In [1]:
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm

In [12]:
import json
import numpy as np

In [3]:
#fpath = "/media/x/data/shape_collection/bkg.bund/vg250_01-01.utm32s.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp"
#fpath = "/media/x/data/kostra/GIS_KOSTRA-DWD-2010R_D0120/StatRR_KOSTRA-DWD-2010R_D0120.shp"
fpath = "/media/x/data/paul/basins_all_info.gpkg"

In [4]:
data = gpd.read_file(fpath)
data = data.to_crs(epsg=4326)

In [5]:
# reduce size
data2 = data[["peak_area_ratio_original", "peak_area_ratio_highest", "highest_peak_event", "geometry"]]
data2[["peak_area_ratio_original", "peak_area_ratio_highest"]] = data2[["peak_area_ratio_original", "peak_area_ratio_highest"]].round(1)

/home/maik/miniconda3/envs/cs2/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
fpath = "/media/x/data/shape_collection/bkg.bund/vg250_01-01.utm32s.shape.ebenen/vg250_ebenen_0101/VG250_LAN.shp"
states = gpd.read_file(fpath)
states = states.to_crs(epsg=4326)

In [40]:
clipdefs = {"nrw": ["Nordrhein-Westfalen"],
            "shh": ["Schleswig-Holstein","Hamburg"],
            "nsb": ["Niedersachsen", "Bremen"],
            "hes": ["Hessen"],
            "rlp": ["Rheinland-Pfalz"],
            "bwb": ["Baden-Württemberg"],
            "bay": ["Bayern"],
            "saa": ["Saarland"],
            "bbr": ["Berlin", "Brandenburg"],
            "mbv": ["Mecklenburg-Vorpommern"],
            "sac": ["Sachsen"],
            "san": ["Sachsen-Anhalt"],
            "thu": ["Thüringen"],
           }

In [44]:
clip = {}
for key in clipdefs.keys():
    print(clipdefs[key])
    if len(clipdefs[key])==1:
        clip[key] = data2.clip(states[states.GEN==clipdefs[key][0]])
    else:
        ix = np.repeat(False, len(states))
        for j, item in enumerate(clipdefs[key]):
            ix = ix | (states.GEN==clipdefs[key][j]).to_numpy()
        clip[key] = data2.clip(states[ix])    

['Nordrhein-Westfalen']
['Schleswig-Holstein', 'Hamburg']
['Niedersachsen', 'Bremen']
['Hessen']
['Rheinland-Pfalz']
['Baden-Württemberg']
['Bayern']
['Saarland']
['Berlin', 'Brandenburg']
['Mecklenburg-Vorpommern']
['Sachsen']
['Sachsen-Anhalt']
['Thüringen']


In [42]:
def _set_precision(coords, precision):
    result = []
    try:
        return round(coords, int(precision))
    except TypeError:
        for coord in coords:
            result.append(_set_precision(coord, precision))
    return result

In [45]:
precision = 5
for key in clip.keys():
    print(key)
    tmp = clip[key].to_json()
    with open("tmp", "w") as f:
        f.write(tmp)
    with open("tmp", "r") as f:
        item = json.load(f)    
    item2 = item.copy()
    for i, feature in enumerate(item["features"]):
        coords = _set_precision(feature['geometry']['coordinates'], precision)
        item2["features"][i]["geometry"]["coordinates"] = coords 
    with open("docs/"+key+".js", "w") as f:
        f.write("%s = " % key)
        json.dump(item2, f)

nrw
shh
nsb
hes
rlp
bwb
bay
saa
bbr
mbv
sac
san
thu


In [17]:
test = np.array([81,23])
test.to

In [ ]:
for key in clip.keys():
    json = clip[key].to_json()
    with open("doc/"+key+".js", "w") as f:
        f.write("cats = ")
        f.write(json)
        #clip[key].to_file(f, driver='GeoJSON', float_format="%.1f")

In [ ]:
with open("doc/Saarland.js", "r") as f:
    tmp = json.load(f)
for feature in tmp["features"]:
    coords = np.round(feature["geometry"]["coordinates"])

In [ ]:
tmp

In [ ]:
test=states.loc[states.index[0], "geometry"]["coords"]

In [ ]:
test.geometry["coords"]

In [ ]:
precision = 5
for ix in states.index:
    coords = np.round(feature.at[ix, 'geometry']['coordinates'], precision)
    states.loc[ix,"geometry"]["coordinates"] = coords
#     if validate:
#         geom = asShape(feature['geometry'])
#         geom.is_valid
#     yield feature

In [ ]:
type(states["geometry"][0])

In [ ]:
x_map=dataclip.centroid.x.mean()
y_map=dataclip.centroid.y.mean()
print(x_map,y_map)

In [ ]:
clip["Nordrhein-Westfalen"]["peak_area_ratio_highest"].hist()

In [ ]:
x_map=tmp.centroid.x.mean()
y_map=tmp.centroid.y.mean()
print(y_map, x_map)

In [ ]:
tmp = clip["Nordrhein-Westfalen"]
tmp["ix"] = tmp.index
x_map=tmp.centroid.x.mean()
y_map=tmp.centroid.y.mean()

mymap = folium.Map(location=[y_map, x_map], zoom_start=6,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)

myscale = (tmp['peak_area_ratio_highest'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()
mymap.choropleth(
 geo_data=tmp,
 name='Choropleth',
 data=tmp,
 columns=['ix', 'peak_area_ratio_highest'],
 key_on="feature.properties.ix",
 fill_color='YlGnBu',
 threshold_scale=myscale,
 fill_opacity=0.8,
 line_opacity=0.2,
)
mymap

In [ ]:
#mymap.save("index.html")

In [ ]:
data[["highest_peak","geometry"]].to_file('data.geojson', driver='GeoJSON', float_format="%.1f")